In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
import pandas as pd

import pickle, os, sys
# imagebind
from imagebind import data
import models.encoder.custom_ib_model as cibm
from models.encoder.custom_ib_model import ModalityType
# mobile_net
import models.classifier.custom_mobile_net as cmn 

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# 데이터 준비 및 모델 로드
df = pd.read_csv('./data/data.csv')

In [ ]:
def get_embeddings(device):
    if os.path.exists('./data/embedbings.pkl'):
        with open('./data/embedbings.pkl', 'rb')as f:
            embeddings = pickle.load(f)
            
        return embeddings
    
    cib_model = cibm.cibm_pretrained(out_embed_dim=1024)
    cib_model.eval()
    cib_model.to(device)

    inputs = {
        ModalityType.TEXT: data.load_and_transform_text(df['portion'], device),
        ModalityType.VISION: data.load_and_transform_vision_data(df['image'], device),
    }
    with torch.no_grad():
        embeddings = cib_model(inputs)

    with open('./data/cib_model.pkl', 'wb')as f:
        pickle.dump(cib_model, f)
    
    with open('./data/embedbings.pkl', 'wb')as f:
        pickle.dump(embeddings, f)

    return embeddings

In [ ]:
embeddings = get_embeddings(device=device)

In [ ]:
def make_labes(touch):
    df2 = pd.read_csv(f'data/{touch}.csv')
    d = [torch.tensor(df2[str(i)].values, dtype=torch.float32) for i in range(1,6)]
    return torch.stack(tuple(d), dim=1)

In [ ]:
# Average the embeddings
features = (embeddings['text'] + embeddings['vision']) / 2
touchs = {
    'softness' : make_labes('softness'),
    'roughness' : make_labes('roughness'),
    'thickness' : make_labes('thickness'),
    'flexibility' : make_labes('flexibility')
}

In [ ]:
# 모델 만들기 
input_size = 1024
models = {}
datasets ={}

for touch, v in touchs.items():
    # Combine features and labels into a single dataset
    datasets[touch] = TensorDataset(features, v)
    models[touch] = cmn.CustomMobileNet(input=1024, num_class = 5)
    models[touch].set_transfer_learn()


In [ ]:
select = 'softness'
model = models[select]
model.to(device)
dataset = datasets[select]

In [ ]:
# 데이터셋 분할
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 모델 학습 및 검증
num_epochs = 25
best_val_loss = float('inf')

In [ ]:
train_loss = []
val_loss = []
if os.path.exists(f'{select}_best_model_{input_size}.pth'):
    model.load_pretrained(f'{select}_best_model_{input_size}.pth')
    

for epoch in range(num_epochs):
    # train, valid loss 계산
    train_loss.append(cmn.train_model(model, train_loader, criterion, optimizer, device, num_epochs=5))
    val_loss.append(cmn.evaluate_model(model, val_loader, criterion, device))
    # valid_loss 최솟값 저장
    if val_loss[-1] < best_val_loss:
        best_val_loss = val_loss[-1]
        print(f'new {best_val_loss}', flush=True)
        torch.save(model.state_dict(), f'{select}_best_model_{input_size}.pth')
    print(f'epoch {epoch} train {train_loss[-1]} valid {val_loss[-1]}', end = '\r', flush=True)
